# Multiple Linear Regression Model

In [1]:
#=============================================================================================
# CODE NAME     : Multiple Linear Regression Model.py
# AUTHOR        : Urvish Shah
# PURPOSE       : Demonstrate application of regression analysis using Pandas and statsmodels 
# APPLICATION   : Analyzing Fitness dataset
#==============================================================================================

In [2]:
# Load Input data
import numpy as np
import pandas as pd

fitness = pd.read_excel("C:\\Users\\Urvish\\Data Science using SAS and Python\\Data\\fitness.xlsx")
fitness.head()

,Name,Gender,RunTime,Age,Weight,Oxygen_Consumption,Run_Pulse,Rest_Pulse,Maximum_Pulse,Performance
0,Donna,F,8.17,42,68.15,59.57,166,40,172,90
1,Gracie,F,8.63,38,81.87,60.06,170,48,186,94
2,Luanne,F,8.65,43,85.84,54.30,156,45,168,83
3,Mimi,F,8.92,50,70.87,54.63,146,48,155,67
4,Chris,M,8.95,49,81.42,49.16,180,44,185,72


# Fit Multiple Linear Regression Model

In [3]:
from statsmodels.formula.api import ols
model = ols('Oxygen_Consumption ~ Performance + RunTime', data = fitness).fit()
model.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Oxygen_Consumption   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     44.09
Date:                Thu, 25 Jun 2020   Prob (F-statistic):           2.23e-09
Time:                        11:16:43   Log-Likelihood:                -73.284
No. Observations:                  31   AIC:                             152.6
Df Residuals:                      28   BIC:                             156.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      71.5263      8.935      8.005      0.000      53.223      89.829
Performance     0.0636      0.047      1.348      0.188      -0.033       0.160
RunTime        -2.6216      0.623     -4.207      0.000      -3.898      -1.345
==============================================================================
Omnibus:                        0.195   Durbin-Watson:                   1.892
Prob(Omnibus):                  0.907   Jarque-Bera (JB):                0.382
Skew:                          -0.130   Prob(JB):                        0.826
Kurtosis:                       2.523   Cond. No.                     1.11e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.11e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [4]:
model.params 

Intercept      71.526257
Performance     0.063596
RunTime        -2.621626
dtype: float64

# Variable Selection by using Rsquare, Adjrsq and Cp 

In [46]:
def get_all_possible_reg(data,X,Y,n_vars):
    
    global all_reg_model,all_reg_model_result
    
    import itertools as itr
    from patsy import ModelDesc, Term, LookupFactor   
    from statsmodels.tools.eval_measures import mse
    
    col_pairs = list(itr.combinations(X.columns,n_vars)) 
    
    all_reg_model = []
    model_params = []
    model_rsquare = []
    model_adj_rsquare = []
    model_mse = []
    
    for cols in col_pairs:        
        formula = str("'" + str(Y + " ~ " + " + ".join(cols)) + "'")    
        response_terms = [Term([LookupFactor(Y)])]
        # start with intercept...
        model_terms = [Term([])]
        # ...then add another term for each predictors
        model_terms += [Term([LookupFactor(c)]) for c in cols]
        model_desc = ModelDesc(response_terms, model_terms)
        
        model = ols(model_desc, data = data).fit()
        
        all_reg_model.append(model)
        model_params.append(" ".join(cols))
        model_rsquare.append(model.rsquared)
        model_adj_rsquare.append(model.rsquared_adj)  
        model_mse.append(mse(data[Y],model.predict(X)))
        
    all_reg_model_result = pd.DataFrame({"Parameters in Model":model_params, "No. of Parameters":n_vars,
                                         "R Square":model_rsquare, "Adj. R Square":model_adj_rsquare,"MSE":model_mse})
    
    return all_reg_model_result

predictors = fitness[['RunTime','Age','Weight','Run_Pulse','Rest_Pulse','Maximum_Pulse','Performance']]


get_all_possible_reg(data = fitness, X = predictors, Y = 'Oxygen_Consumption', n_vars = 4)

# Get the MSE of Full Model
full_model = ols('Oxygen_Consumption ~ RunTime + Age + Weight + Run_Pulse + Rest_Pulse + Maximum_Pulse + Performance', 
                 data = fitness).fit()
all_reg_model_result["MSE_full"] = mse(fitness['Oxygen_Consumption'],full_model.predict(predictors))
all_reg_model_result["Cp"] = all_reg_model_result['No. of Parameters'] + 1 + (((all_reg_model_result['MSE']-all_reg_model_result['MSE_full'])*(len(fitness)-all_reg_model_result['No. of Parameters']+1)) / all_reg_model_result['MSE_full'])
all_reg_model_result

,Parameters in Model,No. of Parameters,R Square,Adj. R Square,MSE,MSE_full,Cp
0,RunTime Age Weight Run_Pulse,4,0.815238,0.786813,5.075329,4.157849,11.178542
1,RunTime Age Weight Rest_Pulse,4,0.775675,0.741163,6.162107,4.157849,18.497181
2,RunTime Age Weight Maximum_Pulse,4,0.786192,0.753299,5.873197,4.157849,16.551587
3,RunTime Age Weight Performance,4,0.777500,0.743269,6.111967,4.157849,18.159524
4,RunTime Age Run_Pulse Rest_Pulse,4,0.810015,0.780787,5.218785,4.157849,12.144609
5,RunTime Age Run_Pulse Maximum_Pulse,4,0.835473,0.810161,4.519474,4.157849,7.435272
6,RunTime Age Run_Pulse Performance,4,0.816480,0.788246,5.041215,4.157849,10.948808
7,RunTime Age Rest_Pulse Maximum_Pulse,4,0.782863,0.749457,5.964655,4.157849,17.167488
8,RunTime Age Rest_Pulse Performance,4,0.778972,0.744968,6.071516,4.157849,17.887116
9,RunTime Age Maximum_Pulse Performance,4,0.790000,0.757693,5.768591,4.157849,15.847141
